In [92]:
import pandas as pd
import janitor
import plotly_express as px
import glob
%matplotlib inline
import numpy as np

In [84]:
train = pd.read_csv('reduce_train.csv')
train.head()
test = pd.read_csv('reduce_test.csv')

In [87]:
test.accuracy_group.index

RangeIndex(start=0, stop=1000, step=1)

In [8]:
train_labels = pd.read_csv('data/train_labels.csv')
train_labels

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.000000,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.500000,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.000000,3
...,...,...,...,...,...,...,...
17685,c996482b11d149dd,ffc90c32,Bird Measurer (Assessment),1,0,1.000000,3
17686,b05a02b52d5c1f4c,ffd2871d,Cauldron Filler (Assessment),1,0,1.000000,3
17687,5448d652309a6324,ffeb0b1b,Cauldron Filler (Assessment),1,2,0.333333,1
17688,a6885ab824fbc32c,ffeb0b1b,Mushroom Sorter (Assessment),0,1,0.000000,0


In [9]:
game_grp = train_labels.groupby(['title','accuracy_group'])['installation_id'].count().reset_index()
display(game_grp)

,title,accuracy_group,installation_id
0,Bird Measurer (Assessment),0,886
1,Bird Measurer (Assessment),1,778
2,Bird Measurer (Assessment),2,389
3,Bird Measurer (Assessment),3,693
4,Cart Balancer (Assessment),0,576
5,Cart Balancer (Assessment),1,353
6,Cart Balancer (Assessment),2,470
7,Cart Balancer (Assessment),3,2752
8,Cauldron Filler (Assessment),0,421
9,Cauldron Filler (Assessment),1,459


In [42]:
from collections import defaultdict

In [82]:
master = defaultdict(dict)
df=train.groupby(['session_title', 'accuracy_group'])['installation_id'].count().T
df = pd.DataFrame(df)
df.reset_index(inplace=True)
for session in df.session_title.unique():
    for ix, row in df.loc[df.session_title==session].iterrows():
        master[session][row.accuracy_group]=row.installation_id
master

defaultdict(dict,
            {8: {0: 576, 1: 353, 2: 470, 3: 2752},
             15: {0: 421, 1: 459, 2: 630, 3: 2545},
             16: {0: 594, 1: 355, 2: 460, 3: 2348},
             20: {0: 1752, 1: 466, 2: 256, 3: 507},
             40: {0: 886, 1: 778, 2: 389, 3: 693}})

In [120]:
cbr_preds = np.array([ 1.98352119,  3.03530927,  2.09923518,  2.40975754,  2.49961708,
        2.72192084,  2.21290432,  2.11656097,  0.52905289,  1.25165101,
        1.63794909,  2.09059268,  2.7846427 ,  1.61509796,  0.67854032,
        1.01489382,  2.00748876,  1.76541535,  2.46143674,  1.86459299,
        2.06507779,  2.08928146,  1.95966796,  2.55177794,  1.71567518,
        0.96743221,  2.63307522,  1.71528581,  1.77864833,  2.33193195,
        0.16483908,  1.85248869,  0.6360326 ,  1.15266514,  2.65429484,
        1.73581515,  0.34099904,  1.83536614,  2.09889564,  0.42517612,
        1.92393992,  2.20918154,  0.21814573,  1.55322922,  2.38501928,
        2.72697653,  1.62269282,  1.24914355,  2.23605457,  2.38031282,
        0.03852631,  1.00747597,  2.67973314,  2.47495046,  2.31970719,
        1.3435652 ,  0.29240625,  2.3915131 ,  1.34870076,  0.84787686,
        1.63284496,  1.56170432,  1.03325574,  2.00296957,  0.55262495,
        2.41107283,  2.25094281,  2.15497596,  1.49751503,  0.38520667,
        1.65704107,  2.08868316,  1.45232049,  1.43735586,  2.14694478,
        2.83323566,  1.78681121,  2.43823276,  1.99300252,  1.50325529,
        1.66674549,  1.6309094 ,  2.20115328,  2.40510239,  2.14244491,
        2.3887251 ,  2.49142071,  0.46034008,  2.35224337,  2.13945502,
        2.49769898,  1.72488701,  2.43879515,  1.52535044, -0.44227459,
        2.38016317,  1.88568848,  1.6153128 ,  0.49263708,  1.85037902,
        2.06757096,  0.17354841,  2.17368643,  1.93860778,  0.66212449,
        2.44276971,  1.65704107,  1.93707939,  2.57748402,  2.54316661,
        1.53239601,  2.55852762,  2.15917861,  1.66854426,  1.56869374,
        1.82053681,  1.78699693,  2.51664571,  1.77052843,  0.75803905,
        2.52713869,  1.93433193,  1.27485169,  2.08097181,  2.67844839,
        0.79802613,  2.39992033,  2.03301765,  2.21565422,  1.97231864,
        2.24905282,  1.2955983 ,  1.62754624,  1.65704107,  2.45374778,
        2.69224984,  2.69386597,  0.71299853,  1.92951827,  2.06333799,
        2.59533879,  2.08091336,  0.80188497,  0.444335  ,  2.39648825,
        2.57798728,  1.17005642,  2.32030187,  2.00075191,  1.51030235,
        1.46502925,  1.01662273,  1.66435002,  0.50749489,  1.87735889,
        0.99586534,  2.46522152,  0.70881441,  1.70386977,  2.98562184,
        1.98869997,  0.59811203,  3.16914784,  2.7850451 ,  1.13422193,
        2.19247348,  2.41352699,  1.45574567,  2.51706862,  1.347488  ,
        1.16336585,  1.11258433,  2.26219528,  2.92848305,  2.08586086,
        2.34351928,  1.85049052,  2.12678445,  0.63712555,  0.79786475,
        1.98272261,  1.10917386,  0.91594791,  0.64163432,  1.98857984,
        1.58841033,  2.92945297,  2.08745885,  2.42199337,  1.78374298,
        3.05630131,  2.09319084,  1.17161645,  1.30036264,  1.61442874,
        2.73603969,  1.81043821,  0.73896936,  1.3360287 ,  2.71554398,
        2.58771958,  2.07899286,  0.68821773, -0.18605577,  2.92220311,
       -0.01158061,  2.15782218, -0.33565131,  2.08998985,  1.65619803,
        2.19097562,  1.41119439,  0.4293936 ,  2.69424504,  2.64420563,
        1.9587481 ,  1.96088958,  1.87854852,  0.49577988,  0.34861332,
        1.80926836,  2.55656828,  1.96820518,  0.15250781,  0.904158  ,
        2.28074149,  2.53883383,  0.20235638,  1.99277404,  1.63657359,
        0.52386247,  2.63700335,  2.40281967,  2.69375468,  2.11580643,
        2.41829578,  1.01986616,  1.63228172,  1.63794608,  1.91743095,
        1.98369456,  0.89161884,  2.10779407,  1.81568861,  2.28497792,
        2.70260464,  1.80334355,  1.99046568,  0.32215726,  1.01049237,
        1.86139859,  2.60228309,  2.39586724,  1.82381249,  2.67212416,
        2.24470838,  1.68368651,  2.1886921 ,  2.51130974,  2.27801452,
        1.03781491,  2.95816605,  2.29744125,  1.2138349 ,  1.81783792,
        1.85596203,  1.90831458,  1.71268409,  0.74133824,  1.49737834,
        0.14040544,  2.39640274,  2.10012318,  0.34481178,  1.08434227,
        1.83388127, -0.05755713,  1.53070692,  2.44741377,  1.75721469,
        2.40039323,  2.63775704,  1.34872609,  1.78771443,  2.00984146,
        2.40588971,  1.57863707,  0.42930903,  1.88878647,  2.32465254,
        2.05252275,  1.88878647,  2.00747919,  1.336909  ,  2.03314371,
        2.28342596,  1.84674925,  2.0393912 ,  1.82243337,  2.64520564,
        0.014033  ,  2.5375468 ,  0.82301857,  1.1121187 ,  2.36613926,
        1.2084819 ,  1.95661922,  2.73121933,  2.51721771,  1.82940458,
        2.24967992,  1.86555499,  2.15929177,  1.43781433,  1.29515561,
        2.50207246,  2.64771559,  1.66317525,  0.29844066,  2.4183229 ,
        2.29939486,  2.4036182 ,  2.62451061,  2.26862459,  2.19725428,
        2.29275045,  2.76994908,  1.68538443,  1.65704107,  1.54788825,
        1.44354238,  0.76011158,  0.57268834,  2.33871024,  1.84172783,
        2.30173608,  2.1378333 ,  1.3963132 ,  1.75071957,  0.52420436,
        2.40074135,  1.48559209,  1.57726179,  1.86590299,  0.87974217,
        2.37915256,  1.82539468,  1.37530384,  1.78880501,  2.22767227,
        1.99823258,  0.51635904,  2.07282415,  2.29750307,  2.82744297,
        2.30723157,  2.81450832,  2.59506715,  1.1647916 ,  2.71375522,
        2.01244706,  2.48342984,  2.12623786,  2.34751661,  1.95394459,
        2.4511765 ,  2.57198611,  2.2489754 ,  2.05641306,  2.70609245,
        1.71450953,  1.5632092 ,  1.62841998,  2.5755839 ,  1.40809406,
        1.9643023 ,  1.48320868,  1.89804358,  2.29490771,  1.80582447,
        2.76963115,  2.60808488,  2.15789006,  0.83496476,  1.81569817,
        0.62135268,  1.82940458,  1.52521733,  1.48656575,  2.06384507,
        1.8150211 ,  1.9434965 ,  1.08727687,  2.80352058,  2.41948058,
        1.46942298,  2.61818356,  1.48193404,  0.65337289,  2.2983    ,
        2.43657256,  2.82806377,  1.41701423,  2.08373729,  1.55971471,
        1.40344162,  1.79278253,  2.26040789,  2.02518972,  2.2490745 ,
        1.75029481,  1.74227662,  2.71689673,  1.94377625,  2.73100915,
        2.07362093,  0.70610163,  2.14569966,  0.72841618,  0.13422947,
        1.86090134,  1.46781315,  2.25082373,  1.23648753,  1.10560116,
        2.47330075,  2.22531891,  2.7143649 ,  1.74268885,  1.40633762,
        2.44525655,  0.47095364,  2.51740841,  0.26378557,  2.38137632,
        2.03673515,  2.85261213,  1.26516071,  0.9105848 ,  2.25952471,
        2.04928381,  0.72500217,  2.13030522,  1.88188022,  0.59826416,
        0.79740326,  0.57129773,  1.73702085,  2.35484509,  0.7158146 ,
        0.96206772,  2.16972961,  2.83364416,  2.16555955,  1.97618247,
        1.66104797,  2.70066638,  1.91964186,  2.17685338,  1.92623608,
        1.59357777,  0.46564147,  2.80192036,  2.21003498,  2.84389459,
        2.16430082,  1.62992449,  2.10579486,  1.29569496,  2.21398949,
        2.54300925,  2.59106307,  2.42685178,  2.15175631,  1.94042612,
        2.12401417,  1.27548266,  2.83070827,  2.09082046,  0.66753979,
        1.86799801,  2.39073928,  2.38205735,  1.91990671,  1.88460316,
        1.04861166,  2.09138677,  2.34852618,  0.61756201,  2.12847024,
        1.77372175,  2.60768319,  1.38886607,  2.14820147,  2.03352902,
        2.58922667,  2.60104488,  2.37381087,  0.2968945 ,  2.38171359,
        2.16326223,  2.19677101,  2.07508958,  2.62588333,  1.91117354,
        1.55053563,  2.55404712,  2.97380101,  2.14375536,  1.37292861,
        0.59260849,  1.75767265,  1.62711795,  2.02529325,  1.35343666,
        1.79878027,  2.37549904,  2.52410528,  2.34012311,  2.70231201,
        3.23371971,  2.35981398,  2.30872056,  1.69424979,  0.68286101,
        1.69165709,  0.93362869,  2.33713849,  1.69439179,  0.92039287,
        2.7031963 ,  1.4453874 ,  0.29396214,  2.2685088 ,  2.112479  ,
        1.39040436,  1.08959403,  2.60495322,  1.6503588 ,  0.47302879,
        3.12307615,  1.8231902 ,  2.79618153,  2.41561685,  2.54031438,
        2.40527342,  1.3664269 ,  1.71450953,  1.96954842,  1.46564335,
        2.02189464,  2.16045322,  2.3404119 ,  0.47136318,  2.72772783,
        1.99769945,  0.20168529,  2.54663259,  1.99730321,  1.82204988,
        1.91089217,  1.41701423,  2.23829821,  0.4636829 ,  3.06941806,
        1.68202935,  1.61723047,  2.126507  ,  1.99481357,  2.463719  ,
        0.05131738,  2.38540373,  1.75078972,  2.73747308,  2.49404638,
        2.30164818,  2.21105458,  1.94206608,  2.16508209,  2.8223002 ,
        0.5519074 ,  2.60762136,  1.35749182,  1.6426346 ,  1.37902259,
        1.23373389,  2.10704849,  1.84599544,  0.80188497,  1.94427247,
        1.65704107,  2.91055836,  2.23328345,  1.48455179,  2.23303033,
        1.96391358,  2.54487519,  2.20848975,  1.50611761,  2.42365618,
        2.64312005,  0.14733653,  1.3758151 ,  1.30079493,  1.57831791,
        1.53197246,  1.13858012,  1.99540459,  2.06416903,  1.94696008,
        1.43148306,  1.86370453,  2.48109791,  0.72034928,  2.37365141,
        2.42946146,  2.36028438,  2.53185946,  1.01110864,  0.69879337,
        2.26921657,  0.88545659,  2.44124819,  1.4771471 ,  2.14763606,
        2.87786087,  0.87601322,  1.46199805,  1.50845564,  2.44574003,
        2.06333372,  2.70883869,  2.20749538,  1.07775085,  2.51113866,
        1.03169349,  1.63473542,  2.08264072,  2.05529   ,  2.45676967,
        0.72314976,  1.92193641,  2.30944577,  1.89195805,  2.41863452,
        1.9196267 ,  1.84805411,  0.14396201,  2.15377818,  2.46270344,
        1.77704398,  1.56488183,  2.06088759,  0.80188497,  2.45842637,
        2.93838997,  2.51751592,  0.3491695 ,  1.86586164,  2.0949629 ,
        1.89326313,  0.85736781,  1.25110413,  2.25352232,  0.3179612 ,
        1.73479203,  1.65704107,  1.18650637,  1.78994548,  2.551189  ,
        2.43036106,  2.30691332,  2.00081656,  2.12778946,  1.44649672,
        1.82539468,  1.97622238,  0.10360814,  1.90243259,  1.05140801,
        1.21169823,  0.90018241,  1.77554012,  1.56170432,  1.5836577 ,
        2.1927796 ,  0.30895184,  1.41701423,  2.31779811,  1.50793201,
        1.01668365,  1.55065294,  2.29087082,  1.36324236,  2.39836513,
        1.76590861,  1.00452748,  1.20271246,  1.98478635,  2.24125559,
        0.93293035,  2.15109023,  2.09287032,  2.4277548 ,  1.34434911,
        2.05570941,  2.3032128 ,  1.5798868 ,  2.93694459,  2.5125471 ,
        2.2795432 ,  2.2093694 ,  1.2409503 ,  1.88743885,  1.51531237,
        2.23127159,  2.19924854,  2.14835024,  2.0252032 ,  2.58415042,
        2.37031137,  2.59966015,  1.72909551,  1.58741593,  2.31357417,
        2.06654499,  2.25571772,  2.82841706,  0.03672605,  2.56655692,
        1.09867325,  1.98516952,  1.11220862, -0.22566021,  0.32991285,
        1.98757258,  1.46230856,  2.65802725,  2.93693253,  2.26199281,
        2.51543939,  2.18004832,  0.91760119,  1.4766582 ,  2.40080757,
        2.0741588 ,  1.61634085,  2.06386815,  1.65633721,  0.4386618 ,
        2.43469756,  1.70008622,  2.82768225,  2.16139221,  1.58656091,
        1.95909265,  2.83020638,  1.9958505 ,  1.82483594,  1.88878647,
        2.02935721,  2.13806142,  1.84732921,  1.98906384,  2.05640653,
        1.31599187,  1.45449298,  2.27463172,  2.23857472,  1.77985642,
        0.36454936,  2.0417579 ,  1.88878647,  1.11747241,  2.29164098,
        0.82471459,  1.18597039,  2.0930808 ,  1.55639815,  2.37294196,
        0.63722959,  0.4585858 ,  2.40699342,  2.64563802,  1.65482394,
        0.80701164,  1.88878647,  2.14164448,  2.06320398,  2.38675847,
        0.86045191,  2.05088569,  2.76798881,  1.55480655,  1.78752479,
        1.81511646,  2.86369661,  1.58728971,  0.40286389,  2.15609754,
        0.69705862,  1.81602856,  1.6279679 ,  1.94054897,  0.18450032,
        1.89710456,  1.7058907 ,  1.13061901,  1.79631194,  2.15355617,
        0.59782573,  0.65896902,  2.44351071,  1.89596758,  2.61125754,
        1.88878647,  2.3078584 ,  2.11688665,  1.71240148,  2.62352565,
        2.09532378,  2.34198126,  2.07797934,  1.61622921,  0.78581683,
        2.87208016,  1.37630791,  2.00075598,  0.86400317,  2.08195365,
        0.86110761,  2.24575589,  2.74478246,  1.65866566,  1.69321007,
        2.57070672,  1.06275042,  1.86459299,  2.13706772,  1.90430409,
        2.92150739,  1.67796638,  1.86363758,  0.80541375,  1.87137311,
        2.30173608,  2.40180279,  2.38516587,  0.69538657,  2.13686981,
        2.2142689 ,  1.82035717,  2.22809187,  3.06434682,  1.52057642,
        1.80907675,  1.62853795,  2.22001001,  1.8851224 ,  1.78945396,
        2.1942591 ,  0.63454364,  1.77512673,  1.96289176,  2.06082408,
        1.88562447,  2.53288135,  2.10829528,  2.05507642,  2.39878111,
        2.74878737,  2.17725828,  2.22883139,  0.68173595,  0.76963624,
        1.70273088,  0.47809183,  1.49214014,  2.24217898,  0.83073727,
        0.06693266,  0.66444905,  2.26477219,  1.70976637,  2.44609056,
        2.725161  ,  2.37956135,  2.4656756 ,  2.60141456,  2.2390297 ,
        1.08995833,  1.65704107,  0.45638842,  1.7605578 ,  0.93731868,
        1.98728815,  2.56277638,  2.2143683 ,  1.49861686,  1.79576864,
        2.15588439,  1.52495795,  2.38059006,  2.52100276,  0.8064474 ,
        1.24558105,  2.14626372,  2.01123283,  1.82436181,  2.30801283,
        0.9385578 ,  0.85698581,  2.55038032,  2.24294197,  1.65847941,
        0.17749893,  2.06076621,  1.95436919,  1.77853733,  2.12917823,
        1.88109662,  1.49612166,  1.78495535, -0.03801288,  1.01904401,
        1.85113698,  2.42595261,  2.82632758,  2.09786831,  0.8607119 ,
        2.20692093,  2.33271929,  1.71207511,  2.27430825,  2.59713169,
        2.20953356,  2.28438677,  0.70240599,  1.36772785,  2.32246141,
        0.56207426,  1.82539468,  1.40917326,  2.14333245,  2.41792596,
        0.84285071,  2.23683334,  2.49640422,  1.60599729,  2.22326283,
        2.89128772,  2.60481282,  2.02103181,  0.97598826,  0.75427171,
        2.88559483,  1.51130274,  1.03844815,  2.44114907,  1.42832282,
        0.26612452,  2.17086575,  2.20008838,  1.16721227,  2.73503232,
        1.85039995,  1.89815702,  2.26071965,  1.47994867,  1.90924729,
        0.6190909 ,  0.76159989,  0.96985826,  1.39746033,  0.95889962,
        1.92222864,  0.85758031,  2.52286511,  2.14257601,  1.91590622,
        2.55846221,  1.79779298,  1.86154441,  2.62297242,  0.6774071 ,
        2.0955901 ,  1.06621139,  2.25293286,  1.1301346 ,  2.9032726 ,
        2.05654086,  2.78917444,  1.54156776,  0.85101217,  1.61685441,
        2.47646993,  1.0321364 ,  2.07008424,  1.58876408,  1.24564139])


In [122]:
test['cbr'] = cbr_preds

## Build class pred function

In [143]:
type(test[test.session_title==8]['accuracy_group'])

pandas.core.series.Series

In [175]:
def get_class_pred(preds, test_df):
    
    test = test_df
    test['accuracy_group'] = preds

    """Example
    {8: {0: 0.9081617833774993, 1: 1.3557792939869913, 2: 1.6557723302312697},
     15: {0: 0.750632477134402, 1: 1.3367320716399504, 2: 1.71574333163995},
     16: {0: 1.0165306932685654, 1: 1.4530646993771625, 2: 1.7328451756800636},
     20: {0: 2.078115677309628, 1: 2.301674688587722, 2: 2.4390138805736328},
     40: {0: 1.6276847459431902, 1: 2.0950951087691188, 2: 2.306488824806992}}
     """
    raw_sd = {8: {0: 576, 1: 353, 2: 470, 3: 2752},
     15: {0: 421, 1: 459, 2: 630, 3: 2545},
     16: {0: 594, 1: 355, 2: 460, 3: 2348},
     20: {0: 1752, 1: 466, 2: 256, 3: 507},
     40: {0: 886, 1: 778, 2: 389, 3: 693}}

    # establish separate bounds for each assessment type (8,15,16,20,40) and assemble master distribution dictionary
    master = {}
    for session in raw_sd:
        session_array = test[test.session_title==session]['accuracy_group']
        dist = raw_sd[session]
        print(f'dist={dist}')
        print(f'dist.values={sum(dist.values())}')
        sum_values = sum(dist.values())
        for k in dist:
            dist[k] /= sum_values
        print(dist)
        acum = 0
        bound = {}
        for i in range(3):
            print(f'acum={acum}')
            acum += dist[i]
            print(f'acum={acum}')
            bound[i] = np.percentile(session_array, acum * 100)
            print(f'{bound}')## <<<< this is where i hit wall 
        master[session] = bound
        print('-'*40)
        print(master)

    def classify(row):
        sid = row['session_title']
        x = row['accuracy_group']
        if x <= master[sid][0]:
            return 0
        elif x <= master[sid][1]:
            return 1
        elif x <= master[sid][2]:
            return 2
        else:
            return 3

    test['accuracy_group'] = test.apply(classify, axis=1)

    return test['accuracy_group']





In [176]:
my_preds = get_class_pred(cbr_preds, test)

dist={0: 576, 1: 353, 2: 470, 3: 2752}
dist.values=4151
{0: 0.13876174415803422, 1: 0.08503974945796193, 2: 0.11322572874006263, 3: 0.6629727776439412}
acum=0
acum=0.13876174415803422
{0: 1.616213276600819}
acum=0.13876174415803422
acum=0.22380149361599616
{0: 1.616213276600819, 1: 1.720755962074199}
acum=0.22380149361599616
acum=0.3370272223560588
{0: 1.616213276600819, 1: 1.720755962074199, 2: 1.92270603393881}
----------------------------------------
{8: {0: 1.616213276600819, 1: 1.720755962074199, 2: 1.92270603393881}}
dist={0: 421, 1: 459, 2: 630, 3: 2545}
dist.values=4055
{0: 0.10382244143033292, 1: 0.11319358816276202, 2: 0.15536374845869297, 3: 0.6276202219482121}
acum=0
acum=0.10382244143033292
{0: 1.7143383556078913}
acum=0.10382244143033292
acum=0.21701602959309496
{0: 1.7143383556078913, 1: 1.88878647}
acum=0.21701602959309496
acum=0.3723797780517879
{0: 1.7143383556078913, 1: 1.88878647, 2: 2.060875766300863}
----------------------------------------
{8: {0: 1.6162132766008

In [178]:
test.accuracy_group.value_counts(normalize=True)

3    0.524
0    0.224
1    0.132
2    0.120
Name: accuracy_group, dtype: float64